# PROYECTO 8

# Introducción

En este proyecto tenemos como objetivo desarrollar un modelo para la compañía Megaline que pueda analizar el comportamiento de sus clientes y recomendar uno de los nuevos planes telefónicos, todo esto con la mayor precisión posible tratando de superar el umbral de exactitud  0.75.

Para este trabajo utilizaremos los datos de comportamiento de los suscriptores que ya se han migrado a alguno de los nuevos planes.


# Carga de Datasets e Importación de Librerias

In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('/datasets/users_behavior_upd.csv')

# Exploración Inicial de Datos

In [3]:
df

,calls,minutes,messages,mb_used,is_ultimate
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [4]:
df.info()
df.head()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   calls        3214 non-null   float64
 1   minutes      3214 non-null   float64
 2   messages     3214 non-null   float64
 3   mb_used      3214 non-null   float64
 4   is_ultimate  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultimate
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


# 

# División de los datos en conjuntos de entrenamiento, validación y prueba

In [32]:
train_ratio = 0.7 
val_ratio = 0.15 
test_ratio = 0.15

train_data, test_data = train_test_split(df, test_size=test_ratio, random_state=42)

train_data, val_data = train_test_split(train_data, test_size=val_ratio/(train_ratio+val_ratio), random_state=42)

print("Tamaño del conjunto de entrenamiento:", len(train_data))
print("Tamaño del conjunto de prueba:", len(test_data))
print("Tamaño del conjunto de validación:", len(val_data))

Tamaño del conjunto de entrenamiento: 2249
Tamaño del conjunto de prueba: 483
Tamaño del conjunto de validación: 482


La división de los datos es importante para garantizar una evaluación confiable del desempeño del modelo evitando el sobreajuste y error; En este código, utilicé la función train_test_split de scikit-learn para dividir los datos en conjuntos de entrenamiento, validación y prueba y establecí la proporción en 70% para capacitación, 15% para validación y 15% para pruebas.

# Formación y evaluación de modelos

In [6]:
df.columns

Index(['calls', 'minutes', 'messages', 'mb_used', 'is_ultimate'], dtype='object')

In [7]:
train_features = train_data.drop("is_ultimate", axis=1)
train_target = train_data["is_ultimate"]

test_features = test_data.drop("is_ultimate", axis=1)
test_target = test_data["is_ultimate"]

val_features = val_data.drop("is_ultimate", axis=1)
val_target = val_data["is_ultimate"]

Antes de comenzar  a probar la calidad y resultados de distintos modelos debemos separar los conjuntos de características en este caso contenidos en todas las columnas excepto is_ultimate de los valores de la columna de destino que es precisamente esta última, de ahí obtenemos train_features, test_features, val_features y train_target, test_target, val_target.

# Investigación de la calidad de diferentes modelos.

Árbol de Decisión

In [35]:
decision_tree = DecisionTreeClassifier()

decision_tree.fit(train_features, train_target)

tree_val_accuracy = decision_tree.score(val_features, val_target)
print("Precisión del modelo de árbol de decisión:", tree_val_accuracy)

Precisión del modelo de árbol de decisión: 0.7240663900414938


Random Forest

In [36]:
random_forest = RandomForestClassifier()

random_forest.fit(train_features, train_target)

forest_val_accuracy = random_forest.score(val_features, val_target)
print("Precisión del modelo de random forest:", forest_val_accuracy)

Precisión del modelo de random forest: 0.7780082987551867


Regresión Logística

In [37]:
logistic_regression = LogisticRegression()

logistic_regression.fit(train_features, train_target)

logistic_val_accuracy = logistic_regression.score(val_features, val_target)
print("Precisión del modelo de regresión logística:", logistic_val_accuracy)

Precisión del modelo de regresión logística: 0.6846473029045643


En base a los resultados podemos observar lo siguiente:

Árbol de decisión: obtuvo un resultado de 0.724 aproximadamente lo cual me parece un buen resultado, sin embargo, no alcanzamos el parámetro mínimo deseado de 0.75 por lo que mas adelante ajustaremos los hiperparámetros para ver si obtenemos un mejor resultado.

Random Forest: de todos fue el que mejor resultado arrojó dando un resultado aproximado de 0.778 cumplindo con el objetivo del 0.75 mínimo pero aún asi igual ajustaremos los hiperparámetros para ver si puede mejorar aun más.

Regresión logística: Fue el que peor resultado arrojo de los 3 por lo cual no creo que sea el modelo indicado, sin embargo, tambíén haremos modificaciones a los hiperparámetros para ver si logramos una mejor precisión.


# Cambio de hiperparámetros

# Árbol de decisión

max_depth: controla la profundidad máxima del árbol y aumentar este valor puede provocar un sobreajuste, mientras que disminuirlo puede provocar un desajuste.

min_samples_split: define el número mínimo de muestras necesarias para dividir un nodo del árbol interno. 

min_samples_leaf: define el número mínimo de muestras necesarias en una hoja del árbol, al aumentar este valor y el valor anterior se puede evitar el sobreajuste.

In [39]:
decision_tree = DecisionTreeClassifier(max_depth=10, min_samples_split=2, min_samples_leaf=1)

decision_tree.fit(train_features, train_target)

tree_val_accuracy = decision_tree.score(val_features, val_target)

print("Precisión del modelo de árbol de decisión:", tree_val_accuracy)

Precisión del modelo de árbol de decisión: 0.7676348547717843


In [12]:
min_samples_leaf_values = [2, 3, 4]
min_samples_split_values = [2, 3, 4]

accuracy_scores = {}

for min_samples_leaf in min_samples_leaf_values:
    
     for min_samples_split in min_samples_split_values:
            
         decision_tree = DecisionTreeClassifier(max_depth=10, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
        
         
         decision_tree.fit(train_features, train_target)
        
         
         val_accuracy = decision_tree.score(val_features, val_target)
        
         
         accuracy_scores[(min_samples_leaf, min_samples_split)] = val_accuracy


for params, accuracy in accuracy_scores.items():
     min_samples_leaf, min_samples_split = params
     print(f"Precisión para min_samples_leaf={min_samples_leaf}, min_samples_split={min_samples_split}: {accuracy}")

Precisión para min_samples_leaf=2, min_samples_split=2: 0.7697095435684648
Precisión para min_samples_leaf=2, min_samples_split=3: 0.7593360995850622
Precisión para min_samples_leaf=2, min_samples_split=4: 0.7697095435684648
Precisión para min_samples_leaf=3, min_samples_split=2: 0.7614107883817427
Precisión para min_samples_leaf=3, min_samples_split=3: 0.7572614107883817
Precisión para min_samples_leaf=3, min_samples_split=4: 0.7614107883817427
Precisión para min_samples_leaf=4, min_samples_split=2: 0.7655601659751037
Precisión para min_samples_leaf=4, min_samples_split=3: 0.7634854771784232
Precisión para min_samples_leaf=4, min_samples_split=4: 0.7614107883817427


Los cambios en los hiperparámetros min_samples_leaf y min_samples_split no dieron como resultado una mejora significativa en la precisión del modelo de árbol de decisión en el conjunto de validación. La precisión más alta obtenida con las combinaciones probadas es 0,7697, pero aún está cerca de la precisión anterior de 0,7676 así que mantendré los valores más altos de los hiperparámetros antes mencionados  y ahora intentaré un bucle for para el hiperparámetro max_ depth.

In [45]:
max_depth_values = [5, 10, 15, 20]

for depth in max_depth_values:
    decision_tree = DecisionTreeClassifier(max_depth=depth, min_samples_split=4, min_samples_leaf=2)
    decision_tree.fit(train_features, train_target)
    val_accuracy = decision_tree.score(val_features, val_target)
    print("Precisión para max_depth={}:".format(depth), val_accuracy)

Precisión para max_depth=5: 0.7406639004149378
Precisión para max_depth=10: 0.7655601659751037
Precisión para max_depth=15: 0.7572614107883817
Precisión para max_depth=20: 0.7344398340248963


El valor max_depth=10 da como resultado la mayor precisión en el conjunto de validación, entonces, para el árbol de decisión mantendré este valor.

# Random Forest

max_features: controla la cantidad de características consideradas en cada división de un nodo.

max_ depth: controla la profundidad máxima de cada árbol en el bosque.

min_samples_split: define el número mínimo de muestras necesarias para dividir un nodo interno de cada árbol.

min_samples_leaf: define el número mínimo de muestras necesarias en una hoja de cada árbol.


In [14]:
random_forest = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=2, min_samples_leaf=1)

random_forest.fit(train_features, train_target)

forest_val_accuracy = random_forest.score(val_features, val_target)

print("Precisión del modelo de random forest:", forest_val_accuracy)

Precisión del modelo de random forest: 0.7780082987551867


La precisión del modelo de bosque aleatorio para clasificación en el conjunto de validación es de 0.7800 aproximadamente pero usaré bucles for para probar diferentes combinaciones de hiperparámetros en el modelo random forest.

In [15]:
min_samples_leaf_values = [2, 3, 4]
min_samples_split_values = [2, 3, 4]

best_accuracy = 0.0
best_min_samples_leaf = None
best_min_samples_split = None

for min_samples_leaf in min_samples_leaf_values:
    for min_samples_split in min_samples_split_values:
        
        random_forest = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

        
        random_forest.fit(train_features, train_target)

        
        forest_val_accuracy = random_forest.score(val_features, val_target)

        print("precisión para min_samples_leaf={}, min_samples_split={}: {}".format(min_samples_leaf, min_samples_split, forest_val_accuracy))

        
        if forest_val_accuracy > best_accuracy:
            best_accuracy = forest_val_accuracy
            best_min_samples_leaf = min_samples_leaf
            best_min_samples_split = min_samples_split

print("\nMejor combinación de hiperparámetros:")
print("min_samples_leaf: ", best_min_samples_leaf)
print("min_samples_split: ", best_min_samples_split)
print("precisión: ", best_accuracy)

precisión para min_samples_leaf=2, min_samples_split=2: 0.7800829875518672
precisión para min_samples_leaf=2, min_samples_split=3: 0.7800829875518672
precisión para min_samples_leaf=2, min_samples_split=4: 0.7800829875518672
precisión para min_samples_leaf=3, min_samples_split=2: 0.7717842323651453
precisión para min_samples_leaf=3, min_samples_split=3: 0.7800829875518672
precisión para min_samples_leaf=3, min_samples_split=4: 0.7780082987551867
precisión para min_samples_leaf=4, min_samples_split=2: 0.7780082987551867
precisión para min_samples_leaf=4, min_samples_split=3: 0.7738589211618258
precisión para min_samples_leaf=4, min_samples_split=4: 0.7821576763485477

Mejor combinación de hiperparámetros:
min_samples_leaf:  4
min_samples_split:  4
precisión:  0.7821576763485477


Ahora probaré diferentes combinaciones de hiperparámetros con n_estimators, max_features y max_ depth.

In [16]:
n_estimators_values = [50, 100, 150]
max_features_values = ['auto', 'sqrt', 'log2']
max_depth_values = [5, 10, 15]

best_accuracy = 0.0
best_n_estimators = None
best_max_features = None
best_max_depth = None

for n_estimators in n_estimators_values:
    for max_features in max_features_values:
        for max_depth in max_depth_values:
            
            random_forest = RandomForestClassifier(n_estimators=n_estimators, max_features=max_features, max_depth=max_depth, min_samples_split=3, min_samples_leaf=4)

            
            random_forest.fit(train_features, train_target)

            
            forest_val_accuracy = random_forest.score(val_features, val_target)

            print("precisión para n_estimators={}, max_features={}, max_depth={}: {}".format(n_estimators, max_features, max_depth, forest_val_accuracy))

            
            if forest_val_accuracy > best_accuracy:
                best_accuracy = forest_val_accuracy
                best_n_estimators = n_estimators
                best_max_features = max_features
                best_max_depth = max_depth

print("\nMejor combinación de hiperparámetros:")
print("n_estimators: ", best_n_estimators)
print("max_features: ", best_max_features)
print("max_depth: ", best_max_depth)
print("precisión: ", best_accuracy)

precisión para n_estimators=50, max_features=auto, max_depth=5: 0.7655601659751037
precisión para n_estimators=50, max_features=auto, max_depth=10: 0.7821576763485477
precisión para n_estimators=50, max_features=auto, max_depth=15: 0.7863070539419087
precisión para n_estimators=50, max_features=sqrt, max_depth=5: 0.7697095435684648
precisión para n_estimators=50, max_features=sqrt, max_depth=10: 0.7800829875518672
precisión para n_estimators=50, max_features=sqrt, max_depth=15: 0.7800829875518672
precisión para n_estimators=50, max_features=log2, max_depth=5: 0.7655601659751037
precisión para n_estimators=50, max_features=log2, max_depth=10: 0.7759336099585062
precisión para n_estimators=50, max_features=log2, max_depth=15: 0.7800829875518672
precisión para n_estimators=100, max_features=auto, max_depth=5: 0.7655601659751037
precisión para n_estimators=100, max_features=auto, max_depth=10: 0.7780082987551867
precisión para n_estimators=100, max_features=auto, max_depth=15: 0.7780082987

Y para asegurar que estos sean los hiperparámetros utilizados a partir de ahora, sustituiré los valores correspondientes en el código del modelo de Random Forest a continuación

In [24]:
random_forest = RandomForestClassifier(n_estimators=25, max_features='auto', max_depth=15, min_samples_split=4, min_samples_leaf=4)


random_forest.fit(train_features, train_target)


forest_val_accuracy = random_forest.score(val_features, val_target)

print("Precisión del modelo de random forest:", forest_val_accuracy)

Precisión del modelo de random forest: 0.7821576763485477


# Regresión Logística

C: será el parámetro de regularización inversa que controla la fuerza de la regularización. 

penalty: que será el tipo de corrección a aplicar y que puede ser "l1" para regularización L1, "l2" para regularización L2 o "ninguno" para no regularización.


In [26]:
logistic_regression = LogisticRegression(C=1.0, penalty='l2')


logistic_regression.fit(train_features, train_target)


logistic_val_accuracy = logistic_regression.score(val_features, val_target)

print("Precisión del modelo de regresión logística:", logistic_val_accuracy)

Precisión del modelo de regresión logística: 0.6846473029045643


# Evaluación de la calidad del modelo

En el código siguiente verificaré la precisión del modelo utilizando el conjunto de validación; random_forest se refiere al modelo random forest ya entrenado, la variable val_features contiene las características del conjunto de validación y val_target contiene las etiquetas correspondientes y por último accuracy_score se utiliza para calcular la precisión del modelo que representa la proporción de predicciones correctas en relación con las predicciones totales.

In [31]:
predictions = random_forest.predict(val_features)

accuracy = accuracy_score(val_target, predictions)
print("Precisión del modelo de random forest:", accuracy)

Precisión del modelo de random forest: 0.7821576763485477


# Prueba del modelo real

Ahora usaré el conjunto de prueba para realizar la prueba real del modelo creado.

In [30]:
predictions = random_forest.predict(test_features)

accuracy = accuracy_score(test_target, predictions)
print("Precisión del modelo de random forest:", accuracy)


Precisión del modelo de random forest: 0.8157349896480331


La precisión del modelo random forest en esta prueba fue aproximadamente del 0.815
 lo que me parece un margen de precisión bueno superando el rango impuesto del 0.75

# Conclusión

A lo largo de este proyecto se ha tratado de dar solución a los cuestionamientos planteados por la empresa Megaline, teniendo como objetivo principal un margen de precisión en el modelo de resultado final superior al 0.75 lo cual se ha intentado conseguir con la prueba de diferentes modelos como el modelo random forest, árbol de decisión y regresión logística, haciendo varias mediciones y pruebas además de cambiar varios hiperparámetros en todos para al final escoger el modelo con mayor precisión posible que superara el objetivo impuesto.

Al final el modelo escogido fue el de random forest porque logró un margen de 0.7821 con los datos del modelo de entrenamiento y un 0.815 en la prueba real del modelo creado, siendo el que mejor resultados logró de todos los modelos probados y superando el objetivo de la empresa Megaline.


